# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [33]:
!pip install nltk
!pip install plotly
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.6 MB/s eta 0:00:00a 0:00:01


In [13]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('omw-1.4') 

[nltk_data] Downloading package punkt to /Users/kobe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kobe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kobe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /Users/kobe/nltk_data...


True

In [3]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from scipy.stats import gmean

import sys
import os
import re
import pickle
from sqlalchemy import create_engine

In [34]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from xgboost import XGBClassifier

In [5]:
# load data from database
engine = create_engine('sqlite:///df_messages_categories.db')
df = df = pd.read_sql_table('df_messages_categories', engine)
X = df['message']
Y = df.iloc[:,4:]

In [6]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [7]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    """
    Tokenise text
    
    Arguments:
        text (str): Text message to be tokenised
    Output:
        list: List of tokens extracted from the text
    """
    
    # Replace all urls with a placeholder string and extract them
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    
    for detected_url in detected_urls:
        text = text.replace(detected_url, "urlplaceholder")

    tokens = nltk.word_tokenize(text) # Extract the word tokens
    
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fa2fb7a0b80>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [29]:
y_pred = pipeline.predict(X_test) # predict on test data

# Evaluate the model
for i, col in enumerate(Y.columns):
    print(f'{col}\n', 
          classification_report(y_test[col], y_pred[:, i], zero_division=0))
    


related
               precision    recall  f1-score   support

           0       0.74      0.27      0.39      1266
           1       0.80      0.97      0.88      3938
           2       0.75      0.15      0.25        40

    accuracy                           0.79      5244
   macro avg       0.76      0.46      0.51      5244
weighted avg       0.78      0.79      0.75      5244

request
               precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.91      0.40      0.56       895

    accuracy                           0.89      5244
   macro avg       0.90      0.70      0.75      5244
weighted avg       0.89      0.89      0.87      5244

offer
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {
    'clf__estimator__n_estimators': [50, 100, 200]# ,
    # 'clf__estimator__min_samples_split': [2, 3, 4],
    # 'tfidf__use_idf': [True, False],
    # 'vect__max_df': [0.75, 1.0, 2],
    # 'vect__ngram_range': [(1, 1), (1, 2)]
}

cv = GridSearchCV(pipeline, param_grid=parameters, 
                  verbose=2, n_jobs=-1, cv=)

In [18]:
# Train the model
cv.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ....................clf__estimator__n_estimators=50; total time= 2.8min
[CV] END ....................clf__estimator__n_estimators=50; total time= 2.8min
[CV] END ....................clf__estimator__n_estimators=50; total time= 2.8min
[CV] END ....................clf__estimator__n_estimators=50; total time= 2.9min
[CV] END ....................clf__estimator__n_estimators=50; total time= 2.9min
[CV] END ...................clf__estimator__n_estimators=100; total time= 6.1min
[CV] END ...................clf__estimator__n_estimators=100; total time= 6.2min
[CV] END ...................clf__estimator__n_estimators=100; total time= 6.2min
[CV] END ...................clf__estimator__n_estimators=100; total time= 6.2min
[CV] END ...................clf__estimator__n_estimators=100; total time= 6.3min
[CV] END ...................clf__estimator__n_estimators=200; total time=10.9min
[CV] END ...................clf__estimator__n_est

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fa2fb7a0b80>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'clf__estimator__n_estimators': [50, 100, 200]},
             verbose=2)

In [19]:
print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf__estimator__n_estimators': 100}


In [ ]:
#finding the best paramesters based on grip search
print(cv.best_params_)

In [20]:
best_model = cv.best_estimator_
print("Best Model:", best_model)

Best Model: Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fa2fb7a0b80>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])


In [21]:
# Predict on test data
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [30]:
# Evaluate the model
for i, col in enumerate(Y.columns):
    print(f'{col}\n', 
          classification_report(y_test[col], y_pred[:, i], zero_division=0))

related
               precision    recall  f1-score   support

           0       0.74      0.27      0.39      1266
           1       0.80      0.97      0.88      3938
           2       0.75      0.15      0.25        40

    accuracy                           0.79      5244
   macro avg       0.76      0.46      0.51      5244
weighted avg       0.78      0.79      0.75      5244

request
               precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.91      0.40      0.56       895

    accuracy                           0.89      5244
   macro avg       0.90      0.70      0.75      5244
weighted avg       0.89      0.89      0.87      5244

offer
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      

In [31]:
# Calculate overall accuracy
accuracy = (y_pred == y_test).mean().mean()
accuracy

0.9456627680311889

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [39]:
# Try other machine learning algorithms

# Define the pipeline with GradientBoostingClassifier
pipeline_gb = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
])

# Define GridSearchCV parameter ranges
parameters_gb = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__learning_rate': [0.1, 0.5]
}
cv_gb = GridSearchCV(pipeline_gb, param_grid=parameters_gb, verbose=2, n_jobs=-1, cv=3)

# Check for classes with only one label in the training data
def remove_single_class_columns(Y):
    cols_to_remove = []
    for col in Y.columns:
        if len(Y[col].value_counts()) < 2:
            cols_to_remove.append(col)
    return Y.drop(columns=cols_to_remove), cols_to_remove
Y_gb, cols_to_remove = remove_single_class_columns(Y)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y_gb, test_size=0.2, random_state=42)

cv_gb.fit(X_train, y_train) # Train the model using GridSearchCV

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50; total time= 3.2min
[CV] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50; total time= 3.3min
[CV] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50; total time= 3.3min
[CV] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50; total time= 3.3min
[CV] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50; total time= 3.3min
[CV] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100; total time= 6.3min
[CV] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100; total time= 6.3min
[CV] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100; total time= 6.4min
[CV] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50; total time= 3.1min
[CV] END clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fa2fb7a0b80>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=GradientBoostingClassifier()))]),
             n_jobs=-1,
             param_grid={'clf__estimator__learning_rate': [0.1, 0.5],
                         'clf__estimator__n_estimators': [50, 100]},
             verbose=2)

In [44]:
best_model_gb = cv_gb.best_estimator_ # Build the best model

# Predict on test data
y_pred_gb = cv_gb.predict(X_test)

# Evaluate the model
for i, col in enumerate(Y_gb.columns): 
    print(f'{col}\n', classification_report(y_test[col], y_pred_gb[:, i], zero_division=0))

# Calculate overall accuracy
accuracy_gb = (y_pred_gb == y_test).mean().mean()
print(f'Overall accuracy with GradientBoostingClassifier: {accuracy_gb:.4f}')

related
               precision    recall  f1-score   support

           0       0.75      0.28      0.41      1266
           1       0.80      0.97      0.88      3938
           2       0.40      0.20      0.27        40

    accuracy                           0.80      5244
   macro avg       0.65      0.48      0.52      5244
weighted avg       0.79      0.80      0.76      5244

request
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.82      0.50      0.62       895

    accuracy                           0.90      5244
   macro avg       0.86      0.74      0.78      5244
weighted avg       0.89      0.90      0.89      5244

offer
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      

In [46]:
# Compare overall accuracy and select the best model
if accuracy_gb > accuracy: 
    best_model = best_model_gb
    print(f"The GradientBoostingClassifier model is selected as the best model, with accuracy {accuracy_gb:.4f}.")
else:
    print(f"The baseline RandomForestClassifier model is selected as the best model, with accuracy {accuracy:.4f}.")


The GradientBoostingClassifier model is selected as the best model, with accuracy 0.9460.


### 9. Export your model as a pickle file

In [47]:
with open('MLclassifier.pkl', 'wb') as file:
    pickle.dump(best_model, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.